<a href="https://colab.research.google.com/github/coderanandmaurya/Feature-engineering/blob/main/column_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import numpy as np
import pandas as pd

In [22]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/coderanandmaurya/Feature-engineering/main/covid_toy.csv')

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [25]:
X_train

,age,gender,fever,cough,city
39,50,Female,103.0,Mild,Kolkata
13,64,Male,102.0,Mild,Bangalore
73,34,Male,98.0,Strong,Kolkata
6,14,Male,101.0,Strong,Bangalore
63,10,Male,100.0,Mild,Bangalore
...,...,...,...,...,...
2,42,Male,101.0,Mild,Delhi
7,20,Female,NaN,Strong,Mumbai
88,5,Female,100.0,Mild,Kolkata
42,27,Male,100.0,Mild,Delhi


# **One by One Encoding**

In [26]:
# adding simple imputer to fever col
si = SimpleImputer(missing_values = np.nan,
                        strategy ='mean')
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])
                                 
X_train_fever

array([[103. ],
       [102. ],
       [ 98. ],
       [101. ],
       [100. ],
       [ 99. ],
       [ 98. ],
       [102. ],
       [100. ],
       [101. ],
       [104. ],
       [101. ],
       [ 99. ],
       [104. ],
       [104. ],
       [100. ],
       [ 98. ],
       [100.9],
       [101. ],
       [ 98. ],
       [ 98. ],
       [102. ],
       [101. ],
       [101. ],
       [104. ],
       [103. ],
       [100. ],
       [ 98. ],
       [100.9],
       [101. ],
       [101. ],
       [ 98. ],
       [104. ],
       [100. ],
       [102. ],
       [ 98. ],
       [103. ],
       [104. ],
       [102. ],
       [ 99. ],
       [ 99. ],
       [100.9],
       [101. ],
       [ 99. ],
       [100.9],
       [ 99. ],
       [ 99. ],
       [104. ],
       [103. ],
       [102. ],
       [100. ],
       [103. ],
       [102. ],
       [ 98. ],
       [101. ],
       [104. ],
       [102. ],
       [ 98. ],
       [104. ],
       [ 98. ],
       [ 98. ],
       [101. ],
       [

In [27]:
X_train_fever.shape

(80, 1)

In [28]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough

array([[0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],

In [29]:
X_train_cough.shape

(80, 1)

In [30]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city

array([[0., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 1., 0.],
       [1., 0., 0., 1.],
       [1., 0., 1., 0.],
       [1., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [1., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [1., 0., 0., 1.],
       [0., 0., 1., 0.],
       [1., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 1., 0., 0.],
       [1., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [1., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],


In [31]:
X_train_gender_city.shape

(80, 4)

In [32]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age

array([[50],
       [64],
       [34],
       [14],
       [10],
       [14],
       [24],
       [74],
       [55],
       [19],
       [ 6],
       [20],
       [65],
       [17],
       [18],
       [13],
       [81],
       [79],
       [49],
       [23],
       [12],
       [33],
       [34],
       [38],
       [44],
       [48],
       [27],
       [40],
       [42],
       [83],
       [64],
       [83],
       [42],
       [11],
       [82],
       [80],
       [69],
       [51],
       [49],
       [59],
       [49],
       [82],
       [51],
       [72],
       [75],
       [60],
       [25],
       [16],
       [70],
       [20],
       [19],
       [60],
       [82],
       [64],
       [19],
       [34],
       [24],
       [10],
       [56],
       [26],
       [73],
       [ 8],
       [65],
       [34],
       [73],
       [15],
       [84],
       [25],
       [11],
       [69],
       [38],
       [71],
       [51],
       [23],
       [54],
       [42],
       [20],

In [33]:
X_train_age.shape

(80, 1)

In [34]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed

array([[ 50. , 103. ,   0. ,   0. ,   1. ,   0. ,   0. ],
       [ 64. , 102. ,   1. ,   0. ,   0. ,   0. ,   0. ],
       [ 34. ,  98. ,   1. ,   0. ,   1. ,   0. ,   1. ],
       [ 14. , 101. ,   1. ,   0. ,   0. ,   0. ,   1. ],
       [ 10. , 100. ,   1. ,   0. ,   0. ,   0. ,   0. ],
       [ 14. ,  99. ,   0. ,   0. ,   0. ,   1. ,   0. ],
       [ 24. ,  98. ,   1. ,   0. ,   1. ,   0. ,   0. ],
       [ 74. , 102. ,   1. ,   0. ,   0. ,   1. ,   0. ],
       [ 55. , 100. ,   1. ,   0. ,   1. ,   0. ,   0. ],
       [ 19. , 101. ,   1. ,   1. ,   0. ,   0. ,   0. ],
       [  6. , 104. ,   0. ,   0. ,   1. ,   0. ,   0. ],
       [ 20. , 101. ,   0. ,   0. ,   0. ,   0. ,   0. ],
       [ 65. ,  99. ,   1. ,   1. ,   0. ,   0. ,   0. ],
       [ 17. , 104. ,   0. ,   0. ,   1. ,   0. ,   0. ],
       [ 18. , 104. ,   0. ,   0. ,   0. ,   0. ,   0. ],
       [ 13. , 100. ,   0. ,   0. ,   1. ,   0. ,   1. ],
       [ 81. ,  98. ,   0. ,   0. ,   0. ,   1. ,   1. ],
       [ 79. ,

In [35]:
X_train_transformed.shape

(80, 7)

# **Using Transformer**

In [36]:
from sklearn.compose import ColumnTransformer

In [37]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(missing_values = np.nan,strategy ='mean'),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [38]:
transformer.fit_transform(X_train)

array([[103. ,   0. ,   0. ,   0. ,   1. ,   0. ,  50. ],
       [102. ,   0. ,   1. ,   0. ,   0. ,   0. ,  64. ],
       [ 98. ,   1. ,   1. ,   0. ,   1. ,   0. ,  34. ],
       [101. ,   1. ,   1. ,   0. ,   0. ,   0. ,  14. ],
       [100. ,   0. ,   1. ,   0. ,   0. ,   0. ,  10. ],
       [ 99. ,   0. ,   0. ,   0. ,   0. ,   1. ,  14. ],
       [ 98. ,   0. ,   1. ,   0. ,   1. ,   0. ,  24. ],
       [102. ,   0. ,   1. ,   0. ,   0. ,   1. ,  74. ],
       [100. ,   0. ,   1. ,   0. ,   1. ,   0. ,  55. ],
       [101. ,   0. ,   1. ,   1. ,   0. ,   0. ,  19. ],
       [104. ,   0. ,   0. ,   0. ,   1. ,   0. ,   6. ],
       [101. ,   0. ,   0. ,   0. ,   0. ,   0. ,  20. ],
       [ 99. ,   0. ,   1. ,   1. ,   0. ,   0. ,  65. ],
       [104. ,   0. ,   0. ,   0. ,   1. ,   0. ,  17. ],
       [104. ,   0. ,   0. ,   0. ,   0. ,   0. ,  18. ],
       [100. ,   1. ,   0. ,   0. ,   1. ,   0. ,  13. ],
       [ 98. ,   1. ,   0. ,   0. ,   0. ,   1. ,  81. ],
       [100.9,

In [39]:
transformer.transform(X_test).shape

(20, 7)